# Lab 5: PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

data = load_iris()
X, y = data.data, data.target

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
val_ds = TensorDataset(X_val_tensor, y_val_tensor)
test_ds = TensorDataset(X_test_tensor, y_test_tensor)


train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)
test_loader = DataLoader(test_ds, batch_size=16)

In [2]:
class IrisNet(nn.Module):
    def __init__(self, hidden1, hidden2, dropout_p=0.5):
        super(IrisNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, hidden1),
            nn.ReLU(),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Linear(hidden2, 3)
        )

    def forward(self, x):
        return self.net(x)

model = IrisNet(hidden1=32, hidden2=16)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model

IrisNet(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [4]:
def evaluate(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.inference_mode():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            probabilities = torch.softmax(outputs, 1) # aktivaciona funkcija
            _, preds = torch.max(probabilities, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    acc = accuracy_score(all_labels, all_preds)
    return acc

In [5]:
evaluate(model, test_loader, device)

0.3333333333333333

In [6]:
import numpy as np

best_val_loss = np.inf
best_val_acc = 0
epochs_no_improve = 0
n_epochs = 50
patience = 5

for epoch in range(n_epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    with torch.inference_mode():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            val_loss += criterion(outputs, y_batch).item() * X_batch.size(0)
    val_loss /= len(val_loader.dataset)
    val_acc = evaluate(model, val_loader, device)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), 'best_iris_model.pth')
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print('Early stopping triggered.')
        break

Early stopping triggered.


In [7]:
model.load_state_dict(torch.load('best_iris_model.pth'))
test_acc = evaluate(model, test_loader, device)
test_acc

0.7333333333333333

In [8]:
output = model(X_test_tensor.to(device))
probabilities = torch.softmax(output, 1)

In [9]:
probabilities

tensor([[0.3026, 0.3334, 0.3640],
        [0.2706, 0.3638, 0.3656],
        [0.2858, 0.3246, 0.3896],
        [0.3112, 0.3642, 0.3247],
        [0.2525, 0.3384, 0.4090],
        [0.6899, 0.1687, 0.1414],
        [0.6620, 0.1897, 0.1483],
        [0.7012, 0.1543, 0.1445],
        [0.2636, 0.3335, 0.4028],
        [0.3120, 0.3265, 0.3615],
        [0.7682, 0.1114, 0.1204],
        [0.2841, 0.3273, 0.3886],
        [0.3297, 0.3187, 0.3516],
        [0.2905, 0.3277, 0.3818],
        [0.7013, 0.1525, 0.1462]], grad_fn=<SoftmaxBackward0>)